In [ ]:
# Complete model usage guidelines available for reference, applicable to the MineMA-8B series models
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = ""

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    device_map = "auto",
    # Or you can specify the device number to use like below
    # device_map = {"": 0},
)

sys_mes = "You are a Large Language Model, and your task is to answer questions posed by users about Minecraft. Utilize your knowledge and understanding of the game to provide detailed, accurate, and helpful responses. Use your capabilities to assist users in solving problems, understanding game mechanics, and enhancing their Minecraft experience."
user_mes = """
How to get diamond in Minecraft?
"""

messages = [
    {"role": "system", "content": sys_mes},
    {"role": "user", "content": user_mes},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]
# If using LLaMA-2-based model, use the following code
"""
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("</s>")
]
"""

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    #repetition_penalty=1.3,  # If there are duplicate problems with model responses, you can use this line of code 
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

In [ ]:
# LoRA model usage guidelines available for reference, applicable to the MineMA-70B series models
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import json
from peft import PeftModel, LoraConfig, TaskType

mode_path = '' # base model path
lora_path = '' # lora model path

tokenizer = AutoTokenizer.from_pretrained(mode_path)

model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16)

with open('config.json', 'r') as f:
    lora_config_data = json.load(f)

config = LoraConfig(**lora_config_data)

model = PeftModel.from_pretrained(model, model_id=lora_path, config=config)

prompt = "How to get diamond in Minecraft?"
messages = [
     {"role": "system", "content": "You are a Large Language Model, and your task is to answer questions posed by users about Minecraft. Utilize your knowledge and understanding of the game to provide detailed, accurate, and helpful responses. Use your capabilities to assist users in solving problems, understanding game mechanics, and enhancing their Minecraft experience."},
    {"role": "user", "content":prompt}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    #repetition_penalty=1.3,  # If there are duplicate problems with model responses, you can use this line of code 
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))